In [17]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
from datetime import datetime, timedelta

# Load the trained model and scaler
with open('gru_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

def predict_and_plot_day(date_str, feature_values):
    """
    Predicts values for each hour of the given day and plots the results.
    
    :param date_str: The date string in 'YYYY-MM-DD' format.
    :param feature_values: A dictionary containing fixed values for the features (excluding datetime).
    """
    # Define the date and time range for the whole day (00:00 to 23:00)
    start_time = datetime.strptime(date_str, '%Y-%m-%d')
    end_time = start_time + timedelta(days=1)
    time_range = pd.date_range(start=start_time, end=end_time - timedelta(hours=1), freq='H')  # End at 23:00

    # Prepare the DataFrame for predictions
    data = []
    for timestamp in time_range:
        row = feature_values.copy()
        row['month'] = timestamp.month
        row['hour'] = timestamp.hour
        row['day'] = timestamp.day
        row['hour_sin'] = np.sin(2 * np.pi * timestamp.hour / 24)
        row['hour_cos'] = np.cos(2 * np.pi * timestamp.hour / 24)
        data.append(row)

    df = pd.DataFrame(data)

    # Ensure all feature columns are present
    feature_columns = ['month', 'hour', 'day', 'hour_sin', 'hour_cos', 'QV2M_toc', 'TQL_toc', 'W2M_toc', 'QV2M_san', 'TQL_san', 'W2M_san', 'QV2M_dav', 'TQL_dav', 'W2M_dav', 'T2M_toc', 'T2M_san', 'T2M_dav', 'holiday']
    df = df[feature_columns]

    # Scale the features
    df_scaled = scaler.transform(df)

    # Reshape the data to fit the model's input shape
    df_reshaped = df_scaled.reshape(df_scaled.shape[0], 1, df_scaled.shape[1])

    # Predict using the loaded model
    predictions = loaded_model.predict(df_reshaped).flatten()

    # Plot the results using Plotly
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=time_range, y=predictions, mode='lines+markers', name='Predictions'))
    fig.update_layout(title=f'Predictions for {date_str}',
                      xaxis_title='Time',
                      yaxis_title='Predicted Value')
    fig.show()

# Example usage
feature_values = {
    'QV2M_toc': 0.018576382,
    'TQL_toc': 0.016174316,
    'W2M_toc': 21.85054581787518,
    'QV2M_san': 0.017271755,
    'TQL_san': 0.0018553734,
    'W2M_san': 10.328948729384228,
    'T2M_toc': 25.865258789062523,
    'QV2M_dav': 0.016562222,
    'TQL_dav': 0.09609985,
    'W2M_dav': 5.364147952093894,
    'T2M_san': 23.482446289062523,
    'T2M_dav': 22.662133789062523,
    'holiday': 0
}

predict_and_plot_day('2015-01-03', feature_values)


1/1 [==============================] - 0s 233ms/step


In [5]:
import requests
import concurrent.futures

def fetch_weather_data(location, start_date, end_date, api_key):
    # Construct the API URL with the custom location, dates, and API key
    url = (
        f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/"
        f"{location}/{start_date}/{end_date}?"
        f"unitGroup=metric&key={api_key}&contentType=json"
    )

    # Make the request to the API
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code != 200:
        return {"error": f"Failed to fetch data for {location}, status code: {response.status_code}"}
    
    return response.json()

def extract_weather_info(data):
    weather_info = {
        "T2M_toc": {},
        "QV2M_toc": {},
        "TQL_toc": {},
        "W2M_toc": {},
        "T2M_san": {},
        "QV2M_san": {},
        "TQL_san": {},
        "W2M_san": {},
        "T2M_dav": {},
        "QV2M_dav": {},
        "TQL_dav": {},
        "W2M_dav": {}
    }

    address_mapping = {
        "Tocumen, Panama": "toc",
        "Santiago, Panama": "san",
        "David, Panama": "dav"
    }

    address = data.get("address")
    location_key = address_mapping.get(address)

    if location_key:
        for day in data.get("days", []):
            for hour in day.get("hours", []):
                hour_time = hour.get("datetime")
                weather_info[f"T2M_{location_key}"][hour_time] = hour.get("temp")
                weather_info[f"QV2M_{location_key}"][hour_time] = hour.get("humidity")
                weather_info[f"TQL_{location_key}"][hour_time] = hour.get("precip")
                weather_info[f"W2M_{location_key}"][hour_time] = hour.get("windspeed")

    return weather_info

def fetch_and_extract_weather_for_all_locations(locations, start_date, end_date, api_key):
    total_weather_info = {}

    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_location = {
            executor.submit(fetch_weather_data, location, start_date, end_date, api_key): location
            for location in locations
        }
        
        for future in concurrent.futures.as_completed(future_to_location):
            location = future_to_location[future]
            try:
                data = future.result()
                weather_info = extract_weather_info(data)
                total_weather_info[location] = weather_info
            except Exception as e:
                print(f"An error occurred for {location}: {e}")

    return total_weather_info

# Custom locations and dates
locations = ["Santiago, Panama", "David, Panama", "Tocumen, Panama"]
start_date = "2024-09-09"
end_date = "2024-09-10"  # Customize end_date as needed

# Your API key
api_key = "7JZFXVY7Q5AHKJC25Z7KJKRDW"

# Fetch and extract weather data for all locations with custom dates
total_weather_info = fetch_and_extract_weather_for_all_locations(locations, start_date, end_date, api_key)

# Print the extracted data for all locations
print(total_weather_info)


{'Santiago, Panama': {'T2M_toc': {}, 'QV2M_toc': {}, 'TQL_toc': {}, 'W2M_toc': {}, 'T2M_san': {'00:00:00': 23.9, '01:00:00': 23.7, '02:00:00': 23.5, '03:00:00': 23.4, '04:00:00': 23.4, '05:00:00': 23.4, '06:00:00': 23.3, '07:00:00': 24.1, '08:00:00': 25.5, '09:00:00': 26.8, '10:00:00': 28.4, '11:00:00': 29.9, '12:00:00': 30.8, '13:00:00': 30.7, '14:00:00': 29.7, '15:00:00': 29.2, '16:00:00': 28.7, '17:00:00': 27.7, '18:00:00': 25.7, '19:00:00': 25.0, '20:00:00': 24.8, '21:00:00': 24.5, '22:00:00': 24.4, '23:00:00': 24.3}, 'QV2M_san': {'00:00:00': 94.71, '01:00:00': 94.71, '02:00:00': 95.28, '03:00:00': 95.27, '04:00:00': 95.27, '05:00:00': 94.69, '06:00:00': 94.69, '07:00:00': 93.02, '08:00:00': 87.65, '09:00:00': 83.15, '10:00:00': 76.63, '11:00:00': 70.69, '12:00:00': 66.73, '13:00:00': 66.31, '14:00:00': 73.24, '15:00:00': 76.3, '16:00:00': 77.14, '17:00:00': 83.25, '18:00:00': 89.27, '19:00:00': 92.5, '20:00:00': 93.05, '21:00:00': 94.17, '22:00:00': 94.16, '23:00:00': 94.73}, 'TQL

In [4]:
import requests

def fetch_weather_data(location, start_date, end_date, api_key):
    url = (
        f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/"
        f"{location}/{start_date}/{end_date}?"
        f"unitGroup=metric&key={api_key}&contentType=json"
    )

    response = requests.get(url)
    
    if response.status_code != 200:
        return None
    
    return response.json()

def process_weather_data(data, location_key):
    weather_dict = {}
    for day in data.get("days", []):
        for hour in day.get("hours", []):
            hour_key = hour.get("datetime", "").split(":")[0] + ":00"  # Format hour
            if hour_key not in weather_dict:
                weather_dict[hour_key] = []
            
            weather_dict[hour_key].append({
                f"T2M_{location_key}": hour.get("temp"),
                f"QV2M_{location_key}": hour.get("humidity"),
                f"TQL_{location_key}": hour.get("precip"),
                f"W2M_{location_key}": hour.get("windspeed")
            })
    
    return weather_dict

def aggregate_weather_data(locations, start_date, end_date, api_key):
    total_weather_info = {}
    
    for location in locations:
        data = fetch_weather_data(location, start_date, end_date, api_key)
        if data:
            location_key = location.split(",")[0].lower()
            location_weather = process_weather_data(data, location_key)
            
            for hour, values in location_weather.items():
                if hour not in total_weather_info:
                    total_weather_info[hour] = [None] * len(locations) * 4  # 4 features per location
                total_weather_info[hour].extend([value for feature in values for value in feature.values()])
    
    return total_weather_info

# Custom locations and dates
locations = ["Santiago, Panama", "David, Panama", "Tocumen, Panama"]
start_date = "2024-09-09"
end_date = "2024-09-10"  # Customize end_date as needed

# Your API key
api_key = "7JZFXVY7Q5AHKJC25Z7KJKRDW"

# Fetch and extract weather data for all locations with custom dates
total_weather_info = aggregate_weather_data(locations, start_date, end_date, api_key)

# Print the aggregated data
print(total_weather_info)


{'00:00': [None, None, None, None, None, None, None, None, None, None, None, None, 23.9, 94.71, 0.0, 7.2, 23.9, 94.71, 0.0, 4.7, 23.6, 91.31, 0.0, 2.5, 23.9, 90.77, 0.0, 1.1, 25.8, 89.27, 0.0, 2.9, 24.4, 91.91, 0.0, 5.8], '01:00': [None, None, None, None, None, None, None, None, None, None, None, None, 23.5, 95.85, 0.0, 6.8, 23.7, 94.71, 0.0, 4.7, 23.3, 92.41, 0.0, 3.6, 23.8, 90.77, 0.0, 2.9, 25.6, 90.34, 0.2, 3.6, 24.3, 92.47, 0.0, 5.4], '02:00': [None, None, None, None, None, None, None, None, None, None, None, None, 23.5, 95.28, 0.0, 6.1, 23.5, 95.28, 0.0, 5.0, 23.1, 92.97, 0.0, 3.6, 23.7, 91.32, 0.0, 2.5, 25.5, 90.33, 0.1, 4.0, 24.1, 92.46, 0.0, 5.4], '03:00': [None, None, None, None, None, None, None, None, None, None, None, None, 23.3, 95.85, 0.0, 5.4, 23.4, 95.27, 0.0, 4.7, 23.1, 92.4, 0.0, 5.8, 23.6, 91.31, 0.0, 1.4, 25.4, 90.33, 0.1, 4.7, 24.0, 93.01, 0.0, 5.0], '04:00': [None, None, None, None, None, None, None, None, None, None, None, None, 23.2, 95.85, 0.0, 5.8, 23.4, 95.27

In [17]:
def fetch_weather_data(location, start_date, end_date, api_key):
    url = (
        f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/"
        f"{location}/{start_date}/{end_date}?"
        f"unitGroup=metric&key={api_key}&contentType=json"
    )
    response = requests.get(url)
    if response.status_code != 200:
        return {"error": f"Failed to fetch data for {location}, status code: {response.status_code}"}
    
    data = response.json()
    return data

def extract_weather_info(data):
    print("extracting data")
    weather_info = {
        "T2M_toc": {},
        "QV2M_toc": {},
        "TQL_toc": {},
        "W2M_toc": {},
        "T2M_san": {},
        "QV2M_san": {},
        "TQL_san": {},
        "W2M_san": {},
        "T2M_dav": {},
        "QV2M_dav": {},
        "TQL_dav": {},
        "W2M_dav": {}
    }

    address_mapping = {
        "Tocumen, Panama": "toc",
        "Santiago, Panama": "san",
        "David, Panama": "dav"
    }

    address = data.get("address")
    location_key = address_mapping.get(address)
    if location_key:
        for day in data.get("days", []):
            for hour in day.get("hours", []):
                hour_time = hour.get("datetime")
                hour_str = hour_time
                weather_info[f"T2M_{location_key}"][hour_str] = hour.get("temp")
                weather_info[f"QV2M_{location_key}"][hour_str] = hour.get("humidity")
                weather_info[f"TQL_{location_key}"][hour_str] = hour.get("precip")
                weather_info[f"W2M_{location_key}"][hour_str] = hour.get("windspeed")

    return weather_info
import requests
import concurrent.futures
import pandas as pd
import numpy as np
import pickle
from flask import Flask, render_template, request
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
from datetime import datetime, timedelta
import keras
def fetch_and_extract_weather_for_all_locations(locations, start_date, end_date, api_key):
    print("fetching data for all locations")
    total_weather_info = {}

    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_location = {
            executor.submit(fetch_weather_data, location, start_date, end_date, api_key): location
            for location in locations
        }

        for future in concurrent.futures.as_completed(future_to_location):
            location = future_to_location[future]
            try:
                data = future.result()
                print("Input data for combining")
                print(data)
                weather_info = extract_weather_info(data)
                total_weather_info[location] = weather_info
            except Exception as e:
                print(f"An error occurred for {location}: {e}")
    return total_weather_info
def flatten_weather_info(total_weather_info):
    flattened_weather_info = {}

    # Iterate through each location's weather data
    for location, weather_data in total_weather_info.items():
        for key, value in weather_data.items():
            # If the key already exists, merge the values
            if key in flattened_weather_info:
                flattened_weather_info[key].update(value)
            else:
                flattened_weather_info[key] = value

    return flattened_weather_info

locations = ["Santiago, Panama", "David, Panama", "Tocumen, Panama"]
api_key = "7JZFXVY7Q5AHKJC25Z7KJKRDW"  # Replace with your actual API key
start_date = "2024-09-09"
end_date = start_date
weather_data = fetch_and_extract_weather_for_all_locations(locations, start_date, end_date, api_key)
weather_data_final=flatten_weather_info(weather_data)
print(weather_data_final)